# Retrieve participant data for Hail GWAS

- runtime: 10min 
- recommended instance: mem1_ssd1_v2_x8
- cost: <£0.10

This notebook depends on:
* **A Spark instance**

In this notebook, we will access phenotypic data stored in the Spark database.

The data is saved as a csv file and uploaded onto the project in a folder called *pheno*.
For the sole purpose of illustrating GWAS in Hail, field 1239 (*Current tobacco smoking*) was considered, this can be modified based on the trait you are interested in.  

## Import `dxdata` package and initialize Spark engine
### Docs at: https://github.com/dnanexus/OpenBio/blob/master/dxdata/getting_started_with_dxdata.ipynb

In [ ]:
import dxdata
import os

# Initialize dxdata engine
engine = dxdata.connect(dialect="hive+pyspark")

## Connect to the dataset

Next, we can set a `DATASET_ID` variable, which takes a value: `[projectID]:[dataset ID]`
We use it to define the `dataset` with `dxdata.load_dataset` function.

**projectID** and **dataset ID** values are unique to your project.
Notebook example **[A101](https://github.com/UK-Biobank/UKB-RAP-Notebooks-Access/blob/main/JupyterNotebook_Python/A101_Explore-phenotype-tables_Python.ipynb)** explains how to get them.

In [ ]:
project = os.popen("dx env | grep project- | awk -F '\t' '{print $2}'").read().rstrip()
record = os.popen("dx describe *dataset | grep  record- | awk -F ' ' '{print $2}'").read().rstrip().split('\n')[0]
DATASET_ID = project + ":" + record
dataset = dxdata.load_dataset(id=DATASET_ID)

## Retrieve data from the table

The following code selects the `participant` table.
Then we can define which field we are interested in using the `find_field` function.
For illustration, we are looking at field 1239 - whether a participant smokes (considering instance 0 - at baseline)

In [ ]:
pheno = dataset['participant']

# Find by field name & title
field_eid = pheno.find_field(name="eid")
smk = pheno.find_field(title="Current tobacco smoking | Instance 0")


In [ ]:
field_list = [field_eid, smk]
field_list

#### Get the data from the Spark DB

In [ ]:
pheno_data = pheno.retrieve_fields(engine=engine, fields=field_list, coding_values="replace")
pheno_data

In [ ]:
data_tab = pheno_data.toPandas()

#### Make the data a binary split
This is in preparation for a basic outline of performing GWAS in Hail - we are not too concerned about providing high quality data (only that it is acceptably formatted for the employed GWAS model)

In [ ]:
data_tab.p1239_i0[data_tab.p1239_i0 != 'No'] = 'true'
data_tab.p1239_i0[data_tab.p1239_i0 == 'No'] = 'false'



In [ ]:
# Set the column names
data_tab.columns = ['eid', 'smoking']

The data is saved as a csv file.   

In [ ]:
data_tab.to_csv('smoking_bool.csv', index=False)

In [ ]:
# the "pheno" folder is created in the project space to upload the csv file created. 
%%bash
dx mkdir pheno
dx upload smok* --path pheno/ # the grep command should be changed based on your file name. 